In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Read into the DataFrame
df = pd.read_csv('../../../../Dropbox/Udemy/100 Days Python/DataAnalysis/Capstone/cleaned_data.csv')

# Drop any stray index column that pandas added
#if 'Unnamed: 0' in df.columns:
#df = df.drop(columns=['Unnamed: 0'])

# Check
print("Loaded", len(df), "rows and", len(df.columns), "columns.")

Loaded 1000 rows and 10 columns.


In [3]:
# Show the first few rows
print("=== Data Preview ===")
display(df.head())

# Show column names + their dtypes
print("\n=== Column Data Types ===")
print(df.dtypes)

# Show descriptive statistics
print("\n=== Data Descriptive Statistics ===")
print(df.describe())

=== Data Preview ===


,Unnamed: 0,Transaction_ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,0,1,24/11/23,CUST001,Male,34.0,Beauty,3.0,50.0,150.0
1,1,2,27/2/23,CUST002,Female,26.0,Clothing,2.0,500.0,1000.0
2,2,3,13/1/23,CUST003,Male,50.0,Electronics,1.0,30.0,30.0
3,4,5,21/5/23,CUST005,Male,37.0,Clothing,1.0,500.0,500.0
4,5,6,6/5/23,CUST006,Male,30.0,Beauty,2.0,50.0,100.0



=== Column Data Types ===
Unnamed: 0            int64
Transaction_ID        int64
Date                 object
Customer ID          object
Gender               object
Age                 float64
Product Category     object
Quantity            float64
Price per Unit      float64
Total Amount        float64
dtype: object

=== Data Descriptive Statistics ===
        Unnamed: 0  Transaction_ID         Age     Quantity  Price per Unit  \
count  1000.000000     1000.000000  1000.00000  1000.000000     1000.000000   
mean    531.146000      532.146000    41.39200     2.514000      179.890000   
std     301.230943      301.230943    13.68143     1.132734      189.681356   
min       0.000000        1.000000    18.00000     1.000000       25.000000   
25%     272.750000      273.750000    29.00000     1.000000       30.000000   
50%     529.500000      530.500000    42.00000     3.000000       50.000000   
75%     793.250000      794.250000    53.00000     4.000000      300.000000   
max    104

In [4]:
# Duplicate Row check & drop
dup_count = df.duplicated().sum()
print(f"Exact duplicate-row count: {dup_count}")
if dup_count > 0:
    df = df.drop_duplicates()
    print(f"Dropped {dup_count} duplicate rows.")

Exact duplicate-row count: 0


In [5]:
# Type Conversions for Dates and Numeric Strings
# reports any rows that fail to parse

date_cols = ['Date']
for c in date_cols:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce')
        n_bad = df[c].isna().sum()
        print(f"{c} parsing failures: {n_bad}")

if 'Price per Unit' in df.columns:
    df['Price per Unit'] = (
        df['Price per Unit'].astype(str)
        .str.replace(r'[$,]', '', regex=True)
        .replace('', pd.NA)
        .astype(float)
    )
    n_bad_price = df['Price per Unit'].isna().sum()
    print(f"Price per Unit conversion failures: {n_bad_price}")

if 'Quantity' in df.columns:
    df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
    n_bad_qty = df['Quantity'].isna().sum()
    print(f"Quantity conversion failures: {n_bad_qty}")

if 'Total Amount' in df.columns:
    df['Total Amount'] = pd.to_numeric(df['Total Amount'], errors='coerce')
    n_bad_total = df['Total Amount'].isna().sum()
    print(f"Total Amount conversion failures: {n_bad_total}")

Date parsing failures: 0
Price per Unit conversion failures: 0
Quantity conversion failures: 0
Total Amount conversion failures: 0


C:\Users\spide\AppData\Local\Temp\ipykernel_13716\1995569080.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors='coerce')


In [6]:
# Total Amount Consistency Check

df = df.copy()

# 2) Compute “Computed_Total” column
df.loc[:, "Computed_Total"] = df["Quantity"] * df["Price per Unit"]

total_col    = "Total Amount"
computed_col = "Computed_Total"
tol          = 1e-6

# 3) Build a boolean mask only over rows where neither value is NaN
valid = df[total_col].notna() & df[computed_col].notna()

# 4) Among those valid rows, check where the absolute difference exceeds tolerance
mismatch = (df.loc[valid, total_col] - df.loc[valid, computed_col]).abs() > tol

# 5) Create a full‐length mask (all False), then assign True where we found mismatches
mismatch_mask = pd.Series(False, index=df.index)
mismatch_mask.loc[valid] = mismatch

print(f"Rows where \"{total_col}\" ≠ \"Quantity × Price per Unit\": {mismatch_mask.sum()}")

if mismatch_mask.sum() > 0:
    display(
        df.loc[
            mismatch_mask,
            ["Quantity", "Price per Unit", "Total Amount", "Computed_Total"]
        ].head()
    )

# 6) Flag those rows for later inspection
df.loc[:, "Amount_Mismatch_Flag"] = mismatch_mask

Rows where "Total Amount" ≠ "Quantity × Price per Unit": 0


In [7]:
# Final Check
# General info (non-null counts, etc.)
print("\nDataFrame info:")
df.info()

# 3) Check for any remaining NaNs (should be zero in required columns)
print("\nMissing values per column:")
missing_counts = df.isna().sum()
missing_percents = (missing_counts / len(df) * 100).round(2)
missing_report = pd.DataFrame({
    "nan_count":   missing_counts,
    "nan_percent": missing_percents
})
display(missing_report)

# 4) Check for any negative or zero values in Quantity, Price per Unit, Total Amount
numeric_cols = ["Quantity", "Price per Unit", "Total Amount"]
neg_zero_report = {}
for col in numeric_cols:
    if col in df.columns:
        neg = (df[col] <= 0).sum()
        neg_zero_report[col] = int(neg)
print("\nCount of Quantity/Price/Total ≤ 0:")
for col, count in neg_zero_report.items():
    print(f"  {col}: {count} rows")


# 5) Check for any Age < 18
if "Age" in df.columns:
    underage = (df["Age"] < 18).sum()
    print(f"\nRows with Age < 18: {underage}")

# 6) Verify Total Amount = Quantity × Price per Unit for every row
qty, price, total = "Quantity", "Price per Unit", "Total Amount"
tol = 1e-6

# Recompute expected total
df["Computed_Total_Check"] = df[qty] * df[price]
valid_mask = df[total].notna() & df["Computed_Total_Check"].notna()

mismatch = (df.loc[valid_mask, total] - df.loc[valid_mask, "Computed_Total_Check"]).abs() > tol
mismatch_mask = pd.Series(False, index=df.index)
mismatch_mask.loc[valid_mask] = mismatch

print(f"\nRows where Total Amount ≠ Quantity × Price per Unit: {mismatch_mask.sum()}")
if mismatch_mask.sum() > 0:
    display(df.loc[mismatch_mask, [qty, price, total, "Computed_Total_Check"]].head())


DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unnamed: 0            1000 non-null   int64         
 1   Transaction_ID        1000 non-null   int64         
 2   Date                  1000 non-null   datetime64[ns]
 3   Customer ID           1000 non-null   object        
 4   Gender                1000 non-null   object        
 5   Age                   1000 non-null   float64       
 6   Product Category      1000 non-null   object        
 7   Quantity              1000 non-null   float64       
 8   Price per Unit        1000 non-null   float64       
 9   Total Amount          1000 non-null   float64       
 10  Computed_Total        1000 non-null   float64       
 11  Amount_Mismatch_Flag  1000 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(5), int64(2), object(

,nan_count,nan_percent
Unnamed: 0,0,0.0
Transaction_ID,0,0.0
Date,0,0.0
Customer ID,0,0.0
Gender,0,0.0
Age,0,0.0
Product Category,0,0.0
Quantity,0,0.0
Price per Unit,0,0.0
Total Amount,0,0.0



Count of Quantity/Price/Total ≤ 0:
  Quantity: 0 rows
  Price per Unit: 0 rows
  Total Amount: 0 rows

Rows with Age < 18: 0

Rows where Total Amount ≠ Quantity × Price per Unit: 0


Additional checks on our cleaned from Nans, negative or zero values data
1. No Duplicates
2. No error in Format type of columns
3. No wrong calculated values on Total Amount
4. Final check to confirm our data include no Nans, negative or zero values and customers with Age < 18
5. Total Percentage of dropped rows: 5%